In [1]:
pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 16.4 MB/s eta 0:00:00


In [2]:
pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.2 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Crippen, rdMolDescriptors, QED
from rdkit.DataStructs import TanimotoSimilarity
import umap

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
path = "/content/drive/My Drive/La_Sorbonne/M2/Meet-U/Final_Code/Similarity Search/Chembl_smiles.csv"

In [24]:
# Read the CSV file into a pandas DataFrame
chembl_df = pd.read_csv(path, delimiter=";", on_bad_lines="skip")  # Use the correct delimiter (';' in this case)

# Display the first few rows of the dataset
chembl_df.head()

,ChEMBL ID,Name,Synonyms,Type,Max Phase,Molecular Weight,Targets,Bioactivities,AlogP,Polar Surface Area,...,Np Likeness Score,Molecular Species,Molecular Formula,Smiles,Inchi Key,Inchi,Withdrawn Flag,Orphan,Records Key,Records Name
0,CHEMBL54661,FLUPENTIXOL,FLUPENTHIXOL|FLUPENTIXOL|LC 44|LC-44|N-7009,Small molecule,3.0,434.53,54.0,143.0,4.60,26.71,...,-0.77,NEUTRAL,C23H25F3N2OS,OCCN1CCN(CC/C=C2/c3ccccc3Sc3ccc(C(F)(F)F)cc32)CC1,NJMYODHXAKYRHW-DVZOWYKESA-N,"InChI=1S/C23H25F3N2OS/c24-23(25,26)17-7-8-22-2...",False,0,"['SID50111069', '36', 'Flupentixol', 'cis-flup...","['SID50111069', '2-(4-{3-[2-Trifluoromethyl-th..."
1,CHEMBL291278,N-METHYL-D-ASPARTIC ACID (NMDA),(R)-2-(Methylamino)Succinic Acid|2-Methylamino...,Small molecule,2.0,147.13,33.0,81.0,-0.87,86.63,...,0.95,ZWITTERION,C5H9NO4,CN[C@H](CC(=O)O)C(=O)O,HOKKHZGPKSLGJE-GSVOUGTGSA-N,"InChI=1S/C5H9NO4/c1-6-3(5(9)10)2-4(7)8/h3,6H,2...",False,0,"['NMDA', 'NMDA', '24', 'NMDA', 'NMDA', 'NMDA',...","['2-Methylamino-succinic acid', '(R)-2-Methyla..."
2,CHEMBL29741,IRAMPANEL,BIIR 561|BIIR-561|BIIR-561 CL|IRAMPANEL|Irampanel,Small molecule,2.0,309.37,3.0,5.0,3.34,51.39,...,-1.54,BASE,C18H19N3O2,CN(C)CCOc1ccccc1-c1nc(-c2ccccc2)no1,QZULPCPLWGCGSL-UHFFFAOYSA-N,InChI=1S/C18H19N3O2/c1-21(2)12-13-22-16-11-7-6...,False,0,"['46', '41', 'IRAMPANEL', 'IRAMPANEL']","['Dimethyl-{2-[2-(3-phenyl-[1,2,4]oxadiazol-5-..."
3,CHEMBL46917,CARBARIL,CARBARIL|CARBARILO|CARBARYL|CARYLDERM|CLINICID...,Small molecule,4.0,201.22,74.0,163.0,2.56,38.33,...,-0.32,NEUTRAL,C12H11NO2,CNC(=O)Oc1cccc2ccccc12,CVXBEEMKQHEXEN-UHFFFAOYSA-N,InChI=1S/C12H11NO2/c1-13-12(14)15-11-8-4-6-9-5...,True,0,"['Carboryl', 'carbaryl', '86', 'Carbaryl 1', '...","['Carboryl', 'Carbaryl', 'naphthalen-1-yl meth..."
4,CHEMBL298359,MITOMYCIN A,6-METHOXY ANALOG OF MITOMYCIN C|MITOMYCIN A|NS...,Small molecule,3.0,349.34,24.0,103.0,-0.96,130.10,...,2.30,NEUTRAL,C16H19N3O6,COC1=C(C)C(=O)C2=C(C1=O)[C@@H](COC(N)=O)[C@@]1...,HYFMSAFINFJTFH-NGSRAFSJSA-N,InChI=1S/C16H19N3O6/c1-6-11(20)10-9(12(21)13(6...,False,0,"['1 (Mytomycin A)', 'mitomycin A', '6', 'mitom...","[6,8a-dimethoxy-5-methyl-4,7-dioxo-(8aR)-1,1a,..."


In [25]:
# remove molecules without smiles
chembl_df = chembl_df[chembl_df['Smiles'].notna() & (chembl_df['Smiles'].str.strip() != '')]

# Molecule Reference

In [28]:
# Reference molecule for Tanimoto similarity
reference_smiles = 'NC1CCC(C(N)O)CN1'
reference_mol = Chem.MolFromSmiles(reference_smiles)
reference_fp = AllChem.GetMorganFingerprintAsBitVect(reference_mol, radius=4, nBits=1024)
reference_properties = [Descriptors.MolWt(reference_mol),  # Molecular Weight
                        Crippen.MolLogP(reference_mol),  # AlogP
                        rdMolDescriptors.CalcTPSA(reference_mol),  # Polar Surface Area
                        rdMolDescriptors.CalcNumHBA(reference_mol),  # HBA
                        rdMolDescriptors.CalcNumHBD(reference_mol),  # HBD
                        rdMolDescriptors.CalcNumLipinskiHBA(reference_mol),  # #RO5 Violations
                        Descriptors.NumRotatableBonds(reference_mol),  # #Rotatable Bonds
                        QED.qed(reference_mol),  # QED Weighted
                        rdMolDescriptors.CalcNumAromaticRings(reference_mol),  # Aromatic Rings
                        int(not reference_mol.HasSubstructMatch(Chem.MolFromSmarts("[!#6]"))),  # Inorganic Flag
                        reference_mol.GetNumHeavyAtoms(),  # Heavy Atoms
                        rdMolDescriptors.CalcNumLipinskiHBA(reference_mol),  # HBA (Lipinski)
                        rdMolDescriptors.CalcNumLipinskiHBD(reference_mol),  # HBD (Lipinski)
                        Chem.rdMolDescriptors.CalcNumLipinskiHBA(reference_mol) > 10,  # #RO5 Violations (Lipinski)
                        Descriptors.ExactMolWt(reference_mol),  # Molecular Weight (Monoisotopic)
                       ]
reference_properties

[10:15:01] DEPRECATION WARNING: please use MorganGenerator


[145.206,
 -1.4520999999999988,
 84.3,
 4,
 4,
 4,
 1,
 0.3400091976066746,
 0,
 0,
 10,
 4,
 6,
 False,
 145.1215121]

## Create new dataframe with ECFP4

In [29]:
# Function to generate ECFP4 fingerprints and compute Tanimoto similarity
def generate_ecfp4_and_tanimoto(smiles, ref_fp, radius=4, nBits=1024):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            ecfp4 = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits)
            tanimoto = TanimotoSimilarity(ref_fp, ecfp4)
            return list(ecfp4), tanimoto
        else:
            return None, None
    except Exception as e:
        print(f"Error processing SMILES: {smiles}, {e}")
        return None, None

# Apply the function to generate ECFP4 fingerprints and Tanimoto similarity
chembl_df['ECFP4'], chembl_df['Tanimoto'] = zip(*chembl_df['Smiles'].apply(lambda x: generate_ecfp4_and_tanimoto(x, reference_fp)))


[10:15:04] DEPRECATION WARNING: please use MorganGenerator
[10:15:04] DEPRECATION WARNING: please use MorganGenerator
[10:15:04] DEPRECATION WARNING: please use MorganGenerator
[10:15:04] DEPRECATION WARNING: please use MorganGenerator
[10:15:04] DEPRECATION WARNING: please use MorganGenerator
[10:15:04] DEPRECATION WARNING: please use MorganGenerator
[10:15:04] DEPRECATION WARNING: please use MorganGenerator
[10:15:04] DEPRECATION WARNING: please use MorganGenerator
[10:15:04] DEPRECATION WARNING: please use MorganGenerator
[10:15:04] DEPRECATION WARNING: please use MorganGenerator
[10:15:04] DEPRECATION WARNING: please use MorganGenerator
[10:15:04] DEPRECATION WARNING: please use MorganGenerator
[10:15:04] DEPRECATION WARNING: please use MorganGenerator
[10:15:04] DEPRECATION WARNING: please use MorganGenerator
[10:15:04] DEPRECATION WARNING: please use MorganGenerator
[10:15:04] DEPRECATION WARNING: please use MorganGenerator
[10:15:04] DEPRECATION WARNING: please use MorganGenerat

In [30]:
print(chembl_df.shape)
chembl_df.head()

(4847, 40)


,ChEMBL ID,Name,Synonyms,Type,Max Phase,Molecular Weight,Targets,Bioactivities,AlogP,Polar Surface Area,...,Molecular Formula,Smiles,Inchi Key,Inchi,Withdrawn Flag,Orphan,Records Key,Records Name,ECFP4,Tanimoto
0,CHEMBL54661,FLUPENTIXOL,FLUPENTHIXOL|FLUPENTIXOL|LC 44|LC-44|N-7009,Small molecule,3.0,434.53,54.0,143.0,4.60,26.71,...,C23H25F3N2OS,OCCN1CCN(CC/C=C2/c3ccccc3Sc3ccc(C(F)(F)F)cc32)CC1,NJMYODHXAKYRHW-DVZOWYKESA-N,"InChI=1S/C23H25F3N2OS/c24-23(25,26)17-7-8-22-2...",False,0,"['SID50111069', '36', 'Flupentixol', 'cis-flup...","['SID50111069', '2-(4-{3-[2-Trifluoromethyl-th...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.029126
1,CHEMBL291278,N-METHYL-D-ASPARTIC ACID (NMDA),(R)-2-(Methylamino)Succinic Acid|2-Methylamino...,Small molecule,2.0,147.13,33.0,81.0,-0.87,86.63,...,C5H9NO4,CN[C@H](CC(=O)O)C(=O)O,HOKKHZGPKSLGJE-GSVOUGTGSA-N,"InChI=1S/C5H9NO4/c1-6-3(5(9)10)2-4(7)8/h3,6H,2...",False,0,"['NMDA', 'NMDA', '24', 'NMDA', 'NMDA', 'NMDA',...","['2-Methylamino-succinic acid', '(R)-2-Methyla...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.045455
2,CHEMBL29741,IRAMPANEL,BIIR 561|BIIR-561|BIIR-561 CL|IRAMPANEL|Irampanel,Small molecule,2.0,309.37,3.0,5.0,3.34,51.39,...,C18H19N3O2,CN(C)CCOc1ccccc1-c1nc(-c2ccccc2)no1,QZULPCPLWGCGSL-UHFFFAOYSA-N,InChI=1S/C18H19N3O2/c1-21(2)12-13-22-16-11-7-6...,False,0,"['46', '41', 'IRAMPANEL', 'IRAMPANEL']","['Dimethyl-{2-[2-(3-phenyl-[1,2,4]oxadiazol-5-...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, ...",0.000000
3,CHEMBL46917,CARBARIL,CARBARIL|CARBARILO|CARBARYL|CARYLDERM|CLINICID...,Small molecule,4.0,201.22,74.0,163.0,2.56,38.33,...,C12H11NO2,CNC(=O)Oc1cccc2ccccc12,CVXBEEMKQHEXEN-UHFFFAOYSA-N,InChI=1S/C12H11NO2/c1-13-12(14)15-11-8-4-6-9-5...,True,0,"['Carboryl', 'carbaryl', '86', 'Carbaryl 1', '...","['Carboryl', 'Carbaryl', 'naphthalen-1-yl meth...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.014493
4,CHEMBL298359,MITOMYCIN A,6-METHOXY ANALOG OF MITOMYCIN C|MITOMYCIN A|NS...,Small molecule,3.0,349.34,24.0,103.0,-0.96,130.10,...,C16H19N3O6,COC1=C(C)C(=O)C2=C(C1=O)[C@@H](COC(N)=O)[C@@]1...,HYFMSAFINFJTFH-NGSRAFSJSA-N,InChI=1S/C16H19N3O6/c1-6-11(20)10-9(12(21)13(6...,False,0,"['1 (Mytomycin A)', 'mitomycin A', '6', 'mitom...","[6,8a-dimethoxy-5-methyl-4,7-dioxo-(8aR)-1,1a,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.085106


In [31]:
# Sort the DataFrame by 'Tanimoto' in descending order
chembl_df_sorted = chembl_df.sort_values(by='Tanimoto', ascending=False)

# Get the top 20 SMILES
top_20_smiles = chembl_df_sorted.head(20)['Smiles'].tolist()

# Display the top 20 SMILES
print("Top 20 SMILES with the highest Tanimoto similarity:")
print(top_20_smiles)

Top 20 SMILES with the highest Tanimoto similarity:
['NC1=NCC(C(=O)O)CN1', 'NC1CCCCC1.NP(=O)(O)NCCC(Cl)Cl', 'NC1CC=C(C(=O)NO)C1', 'NC1=NC(CC(=O)O)CN1', 'N=C(N)NCC1CC1', 'C/C1=C\\CCC(=O)[C@H](C)/C=C(/C(C)(C)O)CC[C@H](C)CC(=O)C1', 'NC1C=CC(C(=O)O)C1', 'C[C@@H]1C[C@H](O)[C@@H](O)CO1', 'CCCC/C=C/C=C/CCC/C=C/C=C/[C@@H]1C[C@H](O)[C@H](N)CN1', 'CC(C)[C@@H](C)/C=C/[C@@H](C)[C@H]1CC[C@@H](C2=C[C@@H]3O[C@@]34C[C@@H](O)CC[C@]4(C)C2=O)[C@]1(C)CC=O', 'CC(C)(O)[C@@H]1CC[C@@]2(C)CCC(C=O)=CC[C@H]12', 'CC1CC1CN=C(N)N', 'Cc1cn([C@H]2C[C@H](O)[C@]3(CO)C[C@H]23)c(=O)[nH]c1=O', 'C=C1CC[C@H]2C[C@@H]1[C@]2(CO)C[C@H](O)/C=C(\\C)C(=O)O', 'Cn1cc(CC(N)CO)cn1', 'NC1CC=C(C(=O)O)C1', 'C=C1CCC[C@H]2[C@](C)(CC/C(C)=C/C[C@@H](O)C3=CC(=O)OC3)[C@@H](C)[C@H](OC(C)=O)C[C@]12C', 'C[C@@H](O)[C@@H](C(=O)O)N1C(=O)C2=C(CCCC2)C1=O', 'CC(C)C(=O)OC1CCC(OC2CCC(OC(=O)C(C)C)CC2)CC1', 'C[C@H]([C@H](O)[C@@H]1C[C@@H](C)C(=O)O1)[C@@H]1[C@H]2[C@]1(C)CC[C@@]13O[C@]2(O)C(=O)[C@@H]1CC[C@H]1C(C)(C)O[C@@H]2[C@@H](O)C(=O)O[C@@]21[C@H]3O']


# Using Machine Learning Model

In [33]:
print('Dataframe Columns :',chembl_df.columns)
# Select numerical features
numerical_features = chembl_df.select_dtypes(include=['number'])
print('Numerical Columns :',numerical_features.columns)

Dataframe Columns : Index(['ChEMBL ID', 'Name', 'Synonyms', 'Type', 'Max Phase',
       'Molecular Weight', 'Targets', 'Bioactivities', 'AlogP',
       'Polar Surface Area', 'HBA', 'HBD', '#RO5 Violations',
       '#Rotatable Bonds', 'Passes Ro3', 'QED Weighted', 'CX Acidic pKa',
       'CX Basic pKa', 'CX LogP', 'CX LogD', 'Aromatic Rings',
       'Structure Type', 'Inorganic Flag', 'Heavy Atoms', 'HBA (Lipinski)',
       'HBD (Lipinski)', '#RO5 Violations (Lipinski)',
       'Molecular Weight (Monoisotopic)', 'Np Likeness Score',
       'Molecular Species', 'Molecular Formula', 'Smiles', 'Inchi Key',
       'Inchi', 'Withdrawn Flag', 'Orphan', 'Records Key', 'Records Name',
       'ECFP4', 'Tanimoto'],
      dtype='object')
Numerical Columns : Index(['Max Phase', 'Molecular Weight', 'Targets', 'Bioactivities', 'AlogP',
       'Polar Surface Area', 'HBA', 'HBD', '#RO5 Violations',
       '#Rotatable Bonds', 'QED Weighted', 'CX Acidic pKa', 'CX Basic pKa',
       'CX LogP', 'CX LogD', 

In [34]:
print(chembl_df.isnull().sum())

ChEMBL ID                             0
Name                               2104
Synonyms                            562
Type                                  3
Max Phase                          4414
Molecular Weight                      0
Targets                             126
Bioactivities                       126
AlogP                               178
Polar Surface Area                  178
HBA                                 178
HBD                                 178
#RO5 Violations                     178
#Rotatable Bonds                    178
Passes Ro3                          178
QED Weighted                        178
CX Acidic pKa                      1700
CX Basic pKa                       3372
CX LogP                             179
CX LogD                             179
Aromatic Rings                      178
Structure Type                        0
Inorganic Flag                        0
Heavy Atoms                         178
HBA (Lipinski)                      178


In [35]:
# We removed Name, Max Phase,CX Acidic pKa, CX Basic pKa
# In this Part we will take into consideration ['Molecular Weight', 'AlogP', 'Polar Surface Area', 'HBA', 'HBD', '#RO5 Violations',
#                                               '#Rotatable Bonds', 'QED Weighted', 'Aromatic Rings', 'Inorganic Flag', 'Heavy Atoms','HBA (Lipinski)',
#                                               'HBD (Lipinski)', '#RO5 Violations (Lipinski)', 'Molecular Weight (Monoisotopic)']

In [36]:
chembl_df.shape

(4847, 40)

In [38]:
dfff = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})

# Add a new row using .loc
dfff.loc[len(dfff)] = {'A': 5, 'B': 6}  # Using a dictionary
# OR
dfff.loc[len(dfff)] = [7, 8]  # Using a list

In [40]:
chembl_df_prop = chembl_df[['Molecular Weight', 'AlogP','Polar Surface Area', 'HBA', 'HBD', '#RO5 Violations','#Rotatable Bonds', 'QED Weighted', 'Aromatic Rings', 'Inorganic Flag', 'Heavy Atoms','HBA (Lipinski)', 'HBD (Lipinski)', '#RO5 Violations (Lipinski)','Molecular Weight (Monoisotopic)']]
chembl_df_prop = chembl_df_prop.dropna()
print(chembl_df_prop.isnull().sum())
chembl_df_prop.loc[len(chembl_df_prop)] = reference_properties
chembl_df_prop.head()

Molecular Weight                   0
AlogP                              0
Polar Surface Area                 0
HBA                                0
HBD                                0
#RO5 Violations                    0
#Rotatable Bonds                   0
QED Weighted                       0
Aromatic Rings                     0
Inorganic Flag                     0
Heavy Atoms                        0
HBA (Lipinski)                     0
HBD (Lipinski)                     0
#RO5 Violations (Lipinski)         0
Molecular Weight (Monoisotopic)    0
dtype: int64


<ipython-input-40-7d4929bdfafd>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  chembl_df_prop.loc[len(chembl_df_prop)] = reference_properties


,Molecular Weight,AlogP,Polar Surface Area,HBA,HBD,#RO5 Violations,#Rotatable Bonds,QED Weighted,Aromatic Rings,Inorganic Flag,Heavy Atoms,HBA (Lipinski),HBD (Lipinski),#RO5 Violations (Lipinski),Molecular Weight (Monoisotopic)
0,434.53,4.60,26.71,4.0,1.0,0.0,5.0,0.63,2.0,0,30.0,3.0,1.0,0.0,434.1640
1,147.13,-0.87,86.63,3.0,3.0,0.0,4.0,0.48,0.0,0,10.0,5.0,3.0,0.0,147.0532
2,309.37,3.34,51.39,5.0,0.0,0.0,6.0,0.70,3.0,0,23.0,5.0,0.0,0.0,309.1477
3,201.22,2.56,38.33,2.0,1.0,0.0,1.0,0.77,2.0,0,15.0,3.0,1.0,0.0,201.0790
4,349.34,-0.96,130.10,8.0,2.0,0.0,4.0,0.49,0.0,0,25.0,9.0,3.0,0.0,349.1274


In [46]:
import umap.umap_ as umap
from sklearn.metrics.pairwise import euclidean_distances


# Separate the last row since it the mol ref
last_row = chembl_df_prop.iloc[-1]

# Preprocess the data
data = chembl_df_prop.iloc[:-1]

# Apply UMAP
reducer = umap.UMAP(n_neighbors=5, min_dist=0.1, n_components=2, random_state=42)
embedding = reducer.fit_transform(data)

# Transform the mol ref
last_row_embedding = reducer.transform([last_row])

# Compute Euclidean distances
distances = euclidean_distances(last_row_embedding, embedding).flatten()

# Find indices of the 20 closest rows
closest_indices = np.argsort(distances)[:20]

# Retrieve the closest rows
closest_rows = chembl_df.iloc[closest_indices]

# Print the results
print("Last row:")
print(last_row)
print("\n20 Closest rows:")
closest_rows['Smiles']

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Last row:
Molecular Weight                    309.27
AlogP                                -3.87
Polar Surface Area                  176.78
HBA                                    8.0
HBD                                    7.0
#RO5 Violations                        1.0
#Rotatable Bonds                       5.0
QED Weighted                          0.27
Aromatic Rings                         0.0
Inorganic Flag                           0
Heavy Atoms                           21.0
HBA (Lipinski)                        10.0
HBD (Lipinski)                         7.0
#RO5 Violations (Lipinski)             1.0
Molecular Weight (Monoisotopic)    309.106
Name: 5102, dtype: object

20 Closest rows:


,Smiles
815,CCCCCCCCCCCC(=O)N/N=C/c1ccccc1
1128,CCCCCCCCCCCCCC(=O)Nc1ccccc1Cl
1529,CN1C(=O)[C@]23SS[C@]1(C)C(=O)N2[C@H]1Nc2ccccc2...
3600,O=C(Nc1cccc(Cl)c1)c1ccccn1
889,NC(=O)NCc1ccc(Cl)cc1
3833,CC(=O)OC[C@@]12[C@@H](OC(C)=O)C[C@@H](C)[C@](C...
3626,COc1cc(/C=C\c2ccc(OC)c(OP(=O)([O-])[O-])c2OP(=...
3929,C=C1C([C@@H](C)[C@H]2CC=C(C)C(=O)O2)=CC[C@]2(C...
1901,CCCCN1CCc2c(c(O)c(OC)c3c2ccc2cc(O)c(OC)cc23)C1
4842,C=C[C@@]1(C)C[C@@H](O)[C@H]2[C@](O)(C1=O)[C@H]...
